## Tools 와 Agent
: LangChain에서 Tools(도구) 는 에이전트(Agent) 가 외부 세계와 상호작용하거나 <br>
특정 기능을 실행하기 위해 사용하는 기능 단위(함수 또는 API 인터페이스) 를 말합니다.

In [2]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
model = ChatOpenAI(model="gpt-4o-mini")

model.invoke([HumanMessage("하늘이 파란 이유는?")])

AIMessage(content='하늘이 파란 이유는 지구의 대기에서 발생하는 산란 현상 때문입니다. 태양빛은 여러 색의 빛으로 구성되어 있으며, 이 중 파란색 빛은 다른 색의 빛보다 파장이 짧습니다. 대기 중의 작은 입자들이 햇빛을 산란시키면서, 파란빛이 특히 많이 산란됩니다. 이로 인해 하늘을 올려다보면 대부분의 경우 파란색으로 보이게 됩니다. 반면, 일몰이나 일출 시에는 태양빛이 대기를 더 많이 통과하면서 주황색이나 빨간색이 강조되어 하늘이 이러한 색으로 보이게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 15, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CUBa538LNOIbP9qv3WQruQarD9nq0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--592b13d6-f17d-4c61-9cd2-947b346f9fba-0', usage_metadata={'input_tokens': 15, 'output_tokens': 148, 'total_tokens': 163, 'input_token_details': {'

### Tools
Tool(도구) 이란 LangChain Agent 가 작업을 수행할 때 실제로 호출할 수 있는 외부 함수 또는 API 기능 이다.<br>
LLM이 “생각(thought)” 단계에서 “이제 무엇을 해야 할까?” 판단했을 때 직접 실행할 수 있는 행동(action)을 정의한 것이다.
https://docs.langchain.com/oss/python/langchain/tools

In [3]:
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from datetime import datetime
import pytz    # 시간대(timezone)를 다루기 위한 모듈

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)  # 특정 지역의 시간대를 객체로 생성
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time
    

# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = model.bind_tools(tools)

# 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

#  llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CUBa8asMI0wLsY8JwZaYyTMpdT69z', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--716ccb12-448c-4388-bf58-ad7b2a8e3eaa-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_hDRB8ShLSs4NxTP76RuVoVg6', 'type': 'tool_call'}],

In [4]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] #  tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"])                     #  도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call)   #  도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-10-24 21:53:47 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CUBa8asMI0wLsY8JwZaYyTMpdT69z', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--716ccb12-448c-4388-bf58-ad7b2a8e3eaa-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_hDRB8ShLSs4NxTP76RuVoVg6', 'type': 'tool_call'}

In [5]:
response = llm_with_tools.invoke(messages)
print(response.content)

부산은 지금 2025년 10월 24일 21시 53분 47초입니다.


### Agent
에이전트란, 고수준의 작업(task)을 받아서 언어모델(LLM)을 추론 엔진(reasoning engine)으로 사용하고,  <br>
적절한 도구(tools)를 선택·실행하며 반복적으로 행동(action) → 관찰(observation) → 추론(thought) 과정을 거쳐 최종 결과를 도출하는 시스템이다.

https://docs.langchain.com/oss/python/releases/langchain-v1

In [14]:
from langchain_openai import ChatOpenAI
# from langgraph.prebuilt import create_react_agent  # Deprecated in LangGraph V1.0 to be removed in V2.0.
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain.tools import tool

# 툴 정의
@tool
def get_weather(city: str) -> str:
    """입력한 도시의 날씨를 알려준다."""
    return f"{city}의 날씨는 맑습니다."
    
@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)  # 특정 지역의 시간대를 객체로 생성
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time
    
# 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Agent 생성
agent = create_agent(llm, [get_weather,get_current_time])

# dict 형태로 메시지를 입력해야 함
result = agent.invoke({
    "messages": [HumanMessage(content="서울의 날씨 알려줘")]
})

# 결과는 messages 리스트 형태로 반환됨
print(result["messages"][-1].content)


서울의 날씨는 맑습니다.


In [15]:
result = agent.invoke({
    "messages": [HumanMessage(content="서울은 지금 몇시야?")]
})

# 결과는 messages 리스트 형태로 반환됨
print(result["messages"][-1].content)

Asia/Seoul (서울) 현재시각 2025-10-24 22:15:32 
서울은 지금 2025년 10월 24일 22시 15분 32초입니다.


In [4]:
# LangChain v1.0 기준 예제
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

# 1) Tool 정의
@tool
def multiply(a: int, b: int) -> int:
    """두 수를 곱한 값을 반환한다."""
    return a * b

@tool
def get_weather(city: str) -> str:
    """예시용 간단 날씨 응답을 반환한다."""
    return f"{city}의 날씨는 맑음이다."

# 2) 모델 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3) 에이전트 생성: create_agent가 표준이다
agent = create_agent(
    model=llm,
    tools=[multiply, get_weather],
    system_prompt="사용자의 요청을 해결하기 위해 필요시 도구를 호출한다."
)

# 4) 호출: v1에서는 메시지 시퀀스를 상태로 전달한다
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "서울의 날씨와 3×7 결과를 알려줘"}
    ]
})
print(result["messages"][-1].content)

서울의 날씨는 흐림입니다. 그리고 3 × 7의 결과는 21입니다.
